In [1]:
import numpy as np
import pandas as pd
import bokeh.plotting as bp
from bokeh.layouts import gridplot

En primera base se probó con la señal del par EUR-USD para ver esta como se comportaba, ahora probaremos con varias señales de entrada, para ver como se comporta la señal EUR-USD.

Para empezar se probará con los pares EUR/USD y GBP/USD para predecir la señal EUR/USD

In [2]:
#Se lee la señal correspondiente a GBP vs USD para el mes de enero de 2018
data_GBPvsUSD = pd.read_csv("../data/TrueFX/GBP-USD/GBPUSD-preprocesado-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

#Se lee la señal correspondiente a EUR vs USD para el mes de enero de 2018
data_EURvsUSD = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_EURUSD-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

In [3]:
data_GBPvsUSD.head()

,bid,ask
date,,
2018-01-02 00:00:40,1.34979,1.35176
2018-01-02 00:01:00,1.34918,1.35208
2018-01-02 00:01:10,1.34915,1.35354
2018-01-02 00:01:20,1.34915,1.35386
2018-01-02 00:01:30,1.34938,1.35440


In [4]:
data_EURvsUSD.head()

,bid,ask
date,,
2018-01-01 23:58:00,1.20093,1.20115
2018-01-01 23:58:10,1.20093,1.20111
2018-01-01 23:58:20,1.20082,1.20111
2018-01-01 23:58:30,1.20043,1.20110
2018-01-01 23:58:50,1.20011,1.20113


In [5]:
print('tamaño de la señal EUR/USD', data_EURvsUSD.shape)
print('tamaño de la señal GBP/USD', data_GBPvsUSD.shape)

tamaño de la señal EUR/USD (183535, 2)
tamaño de la señal GBP/USD (185443, 2)


como se puede ver, las dos señales no tienen el mismo tamaño, como la señal predominante es el par EUR/USD, el indice de este será el tenido en cuenta para la construcción de la multiseñal.

procedemos a cargar la clase "EUtilities" hecha con anterioridad para tratar estos datos...

In [21]:
%run ../code/EUtilities.py
EU = EUtilities()

para poder utilizar la función "multi_signal" se deben cumplir ciertas caracteristicas, entre ellas, que los dataframe a utilizar tenga su columna llamada "bid"...

In [22]:
#se construye el dataset EUR/USD con las dos señales en el, para su posterior uso
data_EURvsUSD = EU.multi_signal(data_EURvsUSD,data_GBPvsUSD)

In [23]:
data_EURvsUSD[:10]

,bid,ask,bidnew
date,,,
2018-01-01 23:58:00,1.20093,1.20115,1.20093
2018-01-01 23:58:10,1.20093,1.20111,1.20093
2018-01-01 23:58:20,1.20082,1.20111,1.20082
2018-01-01 23:58:30,1.20043,1.20110,1.20043
2018-01-01 23:58:50,1.20011,1.20113,1.20011
2018-01-01 23:59:00,1.20015,1.20110,1.20015
2018-01-01 23:59:10,1.20028,1.20110,1.20028
2018-01-01 23:59:50,1.20015,1.20110,1.20015
2018-01-02 00:00:40,1.20015,1.20105,1.34979


ya con la multiseñal se procede a crear el X...

In [24]:
#los datos del par EUR/USD 
X_EURUSD, y_EURUSD, bt_EURUSD = EU.build_dataset(data_EURvsUSD.iloc[:,0],window=3,binary_target=True,PNL=True
                                             ,delete_constant_values=True)
#los datos del par GBP/USD para luego unirlos y formar los datos X para entrenar
X_GBPUSD,_ = EU.build_dataset(data_EURvsUSD.iloc[:,2],window=3,binary_target=False,delete_constant_values=True)

In [25]:
X_EURUSD.shape, X_GBPUSD.shape

((156578, 4), (161098, 3))

debido a que son diferentes señales, al habilitar la opción de <i>"delete_constant_values"</i> en la función <i>"build_dataset"</i> esto hace que las señales resultantes no sean iguales en tamaño, por ende para entrenar se cogeran los valores de los indices de la señal a medir, si este no está en la otra señal, se coloca lo de la señal EUR/USD

In [26]:
X_GBPUSD = X_GBPUSD.loc[X_EURUSD.index]
X_EURUSD.shape, X_GBPUSD.shape

((156578, 4), (156578, 3))

In [27]:
X_GBPUSD[:10]

,0,1,2
date,,,
2018-01-01 23:58:50,1.20093,1.20093,1.20082
2018-01-01 23:59:00,1.20093,1.20082,1.20043
2018-01-01 23:59:10,1.20082,1.20043,1.20011
2018-01-01 23:59:50,1.20043,1.20011,1.20015
2018-01-02 00:00:40,1.20011,1.20015,1.20028
2018-01-02 00:01:00,1.20028,1.20015,1.34979
2018-01-02 00:03:00,1.20030,1.34918,1.34915
2018-01-02 00:03:30,1.34915,1.34940,1.20026
2018-01-02 00:03:40,NaN,NaN,NaN


In [28]:
#para cambiar los valores NaN por los que hay en la señal EUR/USD
X_GBPUSD = X_GBPUSD.fillna(X_EURUSD,axis=0)

se unen para entrenar la multiseñal...

In [29]:
X_EURUSD.insert(loc=3,column=3,value=X_GBPUSD.iloc[:,0])
X_EURUSD.insert(loc=4,column=4,value=X_GBPUSD.iloc[:,1])
X_EURUSD.insert(loc=5,column=5,value=X_GBPUSD.iloc[:,2])

In [30]:
X_EURUSD.head()

,0,1,2,3,4,5,PNL
date,,,,,,,
2018-01-01 23:58:50,1.20093,1.20093,1.20082,1.20093,1.20093,1.20082,-0.00039
2018-01-01 23:59:00,1.20093,1.20082,1.20043,1.20093,1.20082,1.20043,-0.00032
2018-01-01 23:59:10,1.20082,1.20043,1.20011,1.20082,1.20043,1.20011,0.00004
2018-01-01 23:59:50,1.20043,1.20011,1.20015,1.20043,1.20011,1.20015,0.00013
2018-01-02 00:00:40,1.20011,1.20015,1.20028,1.20011,1.20015,1.20028,-0.00013


se prueba con el GaussianNB

In [31]:
%run ../code/PNLEstimatorWrapper.py

In [40]:
from sklearn.naive_bayes import GaussianNB
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column='PNL')
result_GNB = EU.step_validation(est_GNB, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/GaussianNB/pnl-EUR-USD_GBP-USD_GNB_wsize3_2018-01.csv')

--------- GaussianNB ----------------
average PNL :  -0.00102210526316
average buy:  2010.26315789
average sell:  5198.36842105
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05  0.00627   417.0  6041.0          11.0         2864.0
2018-01-08 -0.00614  5424.0  1090.0        4746.0           22.0
2018-01-09 -0.00332  6656.0     0.0        6656.0            0.0
2018-01-10 -0.00100   159.0  6860.0           2.0         1590.0
2018-01-11 -0.00804   130.0  6982.0           2.0         3336.0
2018-01-12 -0.01655     0.0  7093.0           0.0         7093.0
2018-01-15  0.00657  7099.0     0.0        7099.0            0.0
2018-01-16  0.00251   983.0  6139.0           6.0           58.0
2018-01-17  0.00696   841.0  6698.0          17.0           74.0
2018-01-18 -0.00829  1089.0  6269.0          14.0           47.0
2018-01-19  0.00468   928.0  6383.0         160.0         3966.0
2018-01-22  0.00730  1919.0  5232.0          40.0     

se prueba con el KNeighborsClassifier

In [39]:
from sklearn.neighbors import KNeighborsClassifier
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree'), PNL_column='PNL')
result_KNC = EU.step_validation(est_KNC, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('--------- KNeighborsClassifier -------------')
print('average PNL : ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)
df_KNC.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/KNeighborsClassifier/pnl-EUR-USD_GBP-USD_KNC_wsize3_2018-01.csv')

--------- KNeighborsClassifier -------------
average PNL :  -0.000912631578947
average buy:  3360.05263158
average sell:  3848.57894737
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.00021  3175.0  3283.0          60.0          100.0
2018-01-08 -0.00958  4993.0  1521.0         109.0           24.0
2018-01-09 -0.00086  5661.0   995.0        1560.0           28.0
2018-01-10 -0.00346  3850.0  3169.0         202.0           58.0
2018-01-11 -0.00960  4189.0  2923.0          72.0          100.0
2018-01-12 -0.01023   855.0  6238.0          21.0          960.0
2018-01-15 -0.00007  5181.0  1918.0         132.0           54.0
2018-01-16  0.00391  3981.0  3141.0         146.0           61.0
2018-01-17 -0.00060  4791.0  2748.0         132.0          142.0
2018-01-18 -0.00157  2726.0  4632.0          77.0           85.0
2018-01-19 -0.00152  3482.0  3829.0          58.0           60.0
2018-01-22  0.01526  3157.0  3994.0          4

con el DecisionTreeClassifier...

In [47]:
from sklearn.tree import DecisionTreeClassifier
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column='PNL')
result_DTC = EU.step_validation(est_DTC, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('--------- DecisionTreeClassifier -------------')
print('average PNL : ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)
df_DTC.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/DecisionTreeClassifier/pnl-EUR-USD_GBP-USD_DTC_wsize3_2018-01.csv')

--------- DecisionTreeClassifier -------------
average PNL :  -0.000632631578947
average buy:  3254.68421053
average sell:  3953.94736842
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05 -0.00449  2864.0  3594.0          26.0           38.0
2018-01-08  0.00216  1372.0  5142.0          39.0          727.0
2018-01-09 -0.00252  5980.0   676.0        4215.0           19.0
2018-01-10  0.00764  4728.0  2291.0         124.0           32.0
2018-01-11 -0.00416  4211.0  2901.0          42.0           72.0
2018-01-12 -0.01397   773.0  6320.0          12.0          960.0
2018-01-15 -0.00491  1203.0  5896.0          26.0         1187.0
2018-01-16  0.00159  4081.0  3041.0         140.0           27.0
2018-01-17 -0.00160  4690.0  2849.0          91.0           47.0
2018-01-18 -0.00495  3061.0  4297.0          63.0           92.0
2018-01-19 -0.00272  3199.0  4112.0          23.0           80.0
2018-01-22 -0.00642  2814.0  4337.0         

con un RandomForestClassifier...

In [46]:
from sklearn.ensemble import RandomForestClassifier
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column='PNL')
result_RFC = EU.step_validation(est_RFC, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('--------- RandomForestClassifier -------------')
print('average PNL : ', np.mean(result_RFC[:,0]))
print('average buy: ', np.mean(result_RFC[:,1]))
print('average sell: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC)
df_RFC.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/RandomForestClassifier/pnl-EUR-USD_GBP-USD_RFC_wsize3_2018-01.csv')

--------- RandomForestClassifier -------------
average PNL :  -0.00174631578947
average buy:  3507.31578947
average sell:  3701.31578947
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2018-01-05  0.00163  3057.0  3401.0          47.0           32.0
2018-01-08 -0.00872  5023.0  1491.0          71.0           15.0
2018-01-09 -0.00298  5602.0  1054.0        3488.0           37.0
2018-01-10  0.00356  2950.0  4069.0          38.0           35.0
2018-01-11 -0.01260  3646.0  3466.0          36.0           54.0
2018-01-12  0.00395  4332.0  2761.0         117.0          227.0
2018-01-15 -0.00787  4898.0  2201.0         132.0          131.0
2018-01-16  0.00131  4058.0  3064.0         147.0           75.0
2018-01-17  0.00434  5083.0  2456.0          91.0           43.0
2018-01-18 -0.00573  2997.0  4361.0          99.0           52.0
2018-01-19  0.00314  3329.0  3982.0          37.0           43.0
2018-01-22 -0.00902  4290.0  2861.0          